In [3]:
class Account(object):
    #类的静态属性
    interest = .2
    
    def __init__(self, account_holder):
        self.balance = 0
        self.holder = account_holder
    def deposit(self, amount):
        self.balance += amount
    def withdraw(self, amount):
        if amount > self.balance:
            return 'Insufficient funds'
        self.balance -= amount
        return self.balance

###### 点表达式

```
<expression> . <name>
```

1. 求解书点左边的expression, 这会产生点运算符的对象
2. \<name\>会和对象的**实例属性**匹配; 如果该名称的属性存在, 会返回它的值
3. 如果\<name\>不存在于实例属性中, 会在类中查找之, 产生类的**静态属性**
4. 这个植被返回, 如果他是个函数, 会返回绑定方法

###### 赋值
```
<expression> . <name> = abc
```
1. 如果赋值的对象是类的实例, 那么赋值会作用于类的实例属性
2. 如果赋值的对象是类, 那么赋值会作用于类的静态属性

###### 类的静态属性

In [6]:
tom_account = Account('Tom')
jim_account = Account('Jin')

In [8]:
#类的实例可以访问类的静态属性
print(tom_account.interest)
print(jim_account.interest)
#但是如果改变类的静态属性, 类的实例中的该属性也会改变
Account.interest = .8
print('After change Account.interest:', tom_account.interest)
print('After change Account.interest:', jim_account.interest)

0.2
0.2
After change Account.interest: 0.8
After change Account.interest: 0.8


In [9]:
#但是对类的实例赋值不会改变类的静态属性!
tom_account.interest = .5
print('After change tom_account.interest:', tom_account.interest)
print('After change tom_account.interest:', jim_account.interest)

After change tom_account.interest: 0.5
After change tom_account.interest: 0.8


In [13]:
#你会发现 tom_account的__dict__增加了实例属性 interest
print(tom_account.__dict__)
print(jim_account.__dict__)
#print(Account.__dict__)

{'balance': 0, 'holder': 'Tom', 'interest': 0.5}
{'balance': 0, 'holder': 'Jin'}


In [14]:
#同时, tom_account的实例属性会覆盖掉类的静态属性:
Account.interest = .55
print('After change Account.interest:', tom_account.interest)
print('After change Account.interest:', jim_account.interest)

After change Account.interest: 0.5
After change Account.interest: 0.55


###### 继承

In [15]:
class CheckingAccount(Account):
    withdraw_charge = 1
    interest = .01
    def withdraw(self, amount):
        return Account.withdraw(self, amount + self.withdraw_charge)

In [16]:
checking = CheckingAccount('Sam')
checking.interest

0.01

##### 实现类和对象
<u>即使编程语言没有面向对象的系统, 程序照样可以面向对象</u>

In [12]:
TEST = True

###### 实例

In [13]:
def bind_method(value, instance):
    # Return a bound method if value is callable, or value otherwise
    if callable(value):
        if TEST: print(value.__code__.co_varnames)
        def method(*args): 
            return value(instance, *args)
        return method
    else:
        return value
def make_instance(cls_):
    #返回一个新的类的实例, 他是一个 dispatch dict
    def get_value(name):
        #attributes 是类的实例属性字典
        if name in attributes:
            return attributes[name]
        #如果name在attributes字典中不存在, 那么他会在类中寻找。
        #如果cls返回的value是个函数, (在 bind_method中)会与类的实例绑定。
        else:
            value = cls_['get'](name)
            return bind_method(value, instance)
    def set_value(name, value):
        attributes[name] = value
    attributes = {}
    instance = {'get': get_value, 'set': set_value}
    return instance

###### 类
我们假设类自己没有类

In [14]:
def init_instance(cls_, *args):
    #返回类的新的实例, 用args初始化
    instance = make_instance(cls_)
    init = cls_['get']('__init__')
    if init:
        init(instance, *args)
    return instance
def make_class(attributes, base_class=None):
    #返回一个新的类, 他是一个 dispatch dict
    def get_value(name):
        if name in attributes:
            return attributes[name]
        #如果attributes中没有, 就往父类中找
        elif base_class is not None:
            return base_class['get'](name)
    def set_value(name, value):
        attributes[name] = value
    def new(*args):
        return init_instance(cls_, *args)
    cls_ = {'get' : get_value, 'set' : set_value, 'new' : new}
    return cls_

###### 让我们用上面两个函数, 来实现Account类和它的子类CheckingAccount

In [15]:
def make_account_class():
    #这里的self指代可变对象make_account_class的实例
    def __init__(self, account_holder):
        #这里将调用实例的set
        self['set']('holder', account_holder)
        self['set']('balance', 0)
    def deposit(self, amount):
        new_balance = self['get']('balance') + amount
        self['set']('balance', new_balance)
        return self['get']('balance')
    def withdraw(self, amount):
        balance = self['get']('balance')
        if amount > balance:
            return 'Insufficient funds'
        self['set']('balance', balance - amount)
        return self['get']('balance')
    return make_class({
        '__init__' : __init__,
        'deposit' : deposit,
        'withdraw' : withdraw,
        'interest' : .02
    })

In [16]:
Account = make_account_class()

In [17]:
jim_account = Account['new']('Jim')

In [18]:
jim_account['get']('holder')

'Jim'

In [19]:
jim_account['get']('withdraw')(5)

('self', 'amount', 'balance')


'Insufficient funds'

In [20]:
#设置实例的属性不会改变类属性
jim_account['set']('interest', .04)
Account['get']('interest')

0.02

In [22]:
#调用类的set方法, 往类的attributes中加变量
Account['set']('mamami', 10)

In [23]:
#实例的set方法只会新加实例属性, get方法首先查找实例属性(比如在__init__中定义的)字典,要是找不到就再到类的attributes中找,
#而类的attributes中包括类方法和雷属性
jim_account['get']('mamami')

10

In [42]:
jim_account

{'get': <function __main__.make_instance.<locals>.get_value(name)>,
 'set': <function __main__.make_instance.<locals>.set_value(name, value)>}

In [43]:
jim_account['get']('interest')

0.04

In [65]:
#子类
def make_checking_account_class():
    def withdraw(self, amount):
        if TEST: print(self)
        return Account['get']('withdraw')(self, amount + 1)
    return make_class({'withdraw' : withdraw, 'interest' : .01},
                      Account
                     )
    

In [66]:
CheckingAccount = make_checking_account_class()
jack_account = CheckingAccount['new']('Jack')

In [67]:
jack_account['get']('withdraw')

('self', 'amount')


<function __main__.bind_method.<locals>.method(*args)>

In [68]:
jack_account['get']('withdraw')(5)

('self', 'amount')
{'get': <function make_instance.<locals>.get_value at 0x7f1ba8f5a598>, 'set': <function make_instance.<locals>.set_value at 0x7f1ba8f58620>}


'Insufficient funds'

##### 泛用函数

In [72]:
#我们首先定义两种复数类型
#1. 实部和虚部来表示复数
from math import atan2
class ComplexRI(object):
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag
    @property
    def magnitude(self):
        return (self.real ** 2 + self.imag ** 2) ** (1/2)
    @property
    def angle(self):
        return atan2(self.imag, self.real)
    def __repr__(self):
        return 'ComplexRI({0}, {1})'.\
               format(self.real, self.imag)

In [74]:
#2. 模和幅角来表示复数
from math import sin, cos
class ComplexMA(object):
    def __init__(self, magnitude, angle):
        self.magnitude = magnitude
        self.angle = angle
    @property
    def real(self):
        return self.magnitude * cos(self.angle)
    @property
    def imag(self):
        return self.magnitude * sin(self.angle)
    def __repr__(self):
        return 'ComplexMA({0}, {1})'.\
                format(self.magnitude, self.angle)

###### 我们首先实现两个接口: add_complex 和 mul_complex

In [78]:
#这两个接口由 ComplexMA和 ComplexRI共享,
#他们是复数类们的通用接口
def add_complex(z1, z2):
    return ComplexRI(z1.real + z2.real, z1.imag + z2.imag)
def mul_complex(z1, z2):
    return ComplexMA(z1.magnitude * z2.magnitude, z1.angle + z2.angle)

In [79]:
from math import pi
add_complex(ComplexRI(1, 2), ComplexMA(2, pi/2))

ComplexRI(1.0000000000000002, 4.0)

In [80]:
#当然为了使我们的代码更加易读, 我们可以在复数类里面实现魔法函数
#我们把全部的代码放在这里
def add_complex(z1, z2):
    return ComplexRI(z1.real + z2.real, z1.imag + z2.imag)
def mul_complex(z1, z2):
    return ComplexMA(z1.magnitude * z2.magnitude, z1.angle + z2.angle)


from math import atan2
class ComplexRI(object):
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag
    @property
    def magnitude(self):
        return (self.real ** 2 + self.imag ** 2) ** (1/2)
    @property
    def angle(self):
        return atan2(self.imag, self.real)
    def __repr__(self):
        return 'ComplexRI({0}, {1})'.\
               format(self.real, self.imag)
    def __add__(self, other):
        return add_complex(self, other)
    def __mul__(self, other):
        return mul_complex(self, other)

from math import sin, cos
class ComplexMA(object):
    def __init__(self, magnitude, angle):
        self.magnitude = magnitude
        self.angle = angle
    @property
    def real(self):
        return self.magnitude * cos(self.angle)
    @property
    def imag(self):
        return self.magnitude * sin(self.angle)
    def __repr__(self):
        return 'ComplexMA({0}, {1})'.\
                format(self.magnitude, self.angle)
    def __add__(self, other):
        return add_complex(self, other)
    def __mul__(self, other):
        return mul_complex(self, other)

In [83]:
print(ComplexRI(1, 2) + ComplexMA(2, 0))
print(ComplexRI(0, 1) * ComplexRI(0, 1))

ComplexRI(3.0, 2.0)
ComplexMA(1.0, 3.141592653589793)


###### 我们关于复数的两个接口仍然不完善: 数学上复数可以跟有理数相加但我们还没有实现

In [84]:
#我们将实现一个有理数类
from fractions import gcd
class Rational(object):
    def __init__(self, numer, denom):
        g = gcd(numer, denom)
        self.numer = numer // g
        self.denom = denom // g
    def __repr__(self):
        return 'Rational {0} / {1}'.\
               format(self.numer, self.denom)

In [85]:
def add_rational(x, y):
    nx, ny = x.numer, y.numer
    dx, dy = x.denom, y.denom
    return Rational(nx * dy + ny * dx, dx * dy)
def mul_rational(x, y):
    return Rational(x.numer * y.numer, x.denom * y.denom)    

In [87]:
#我们来实现跨类型相加和相乘函数
#一种典型的方法叫类型分派
def is_complex(z):
    return isinstance(z, ComplexMA) or isinstance(z, ComplexRI)
def is_rational(z):
    return isinstance(z, Rational)
#这样我们实现一个更一般化的add函数
def add_complex_and_rational(z, r):
    return ComplexRI(z.real + r.numer / r.denom, z.imag)

def add(z1, z2):
    if is_complex(z1) and is_complex(z2):
        return add_complex(z1, z2)
    elif is_complex(z1) and is_rational(z2):
        return add_complex_and_rational(z1, z2)
    elif is_rational(z1) and is_complex(z2):
        return add_complex_and_rational(z2, z1)
    elif is_rational(z1) and is_rational(z2):
        return add_rational(z1, z2)
    else:
        raise ValueError   

In [91]:
#当然我们也可以用字典代替如此繁复的 if-else语句
type_tags = {ComplexRI : 'com', ComplexMA : 'com', Rational : 'rat'}
def type_tag(x):
    return type_tags[type(x)]
add_implementations = {
    ('com', 'com') : add_complex,
    ('com', 'rat') : add_complex_and_rational,
    ('rat', 'com') : lambda x, y : add_complex_and_rational(y, x),
    ('rat', 'rat') : add_rational
}
def add(z1, z2):
    types = (type_tag(z1), type_tag(z2))
    return add_implementations[types](z1, z2)

In [92]:
add(ComplexRI(1.5, 0), Rational(3, 2))

/home/zhangzijie_dlut/miniconda3/envs/zzj/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: fractions.gcd() is deprecated. Use math.gcd() instead.
  """


ComplexRI(3.0, 0)

In [97]:
#我们还可以将加法运算和乘法运算合并为一起
#apply函数支持任何类型的任何算数操作
apply_implementations = dict()
def apply(operator_name, x, y):
    tags = (type_tag(x), type_tag(y))
    key = (operator_name, tags)
    return apply_implementations[key](x, y)

In [95]:
#我们把add_implemantations的方法加到 apply_implementations中去
apply_implementations.update(
    {('add', tags): func for (tags, func) in add_implementations.items()}
)

In [96]:
apply_implementations

{('add', ('com', 'com')): <function __main__.add_complex(z1, z2)>,
 ('add', ('com', 'rat')): <function __main__.add_complex_and_rational(z, r)>,
 ('add', ('rat', 'com')): <function __main__.<lambda>(x, y)>,
 ('add', ('rat', 'rat')): <function __main__.add_rational(x, y)>}

使用上面的**数据导向**的系统有效的管理了跨类型运算的复杂性.<br>
但是维护起来比较麻烦, 比如在引入新类型时, 需要实现跨类型操作的函数的构造和安装.<br>
我们可以使用**强制类型转换**

###### 类型转换
这样我们可以为每对类型编写一个函数而不是为每个类型的组合编写

In [99]:
def rational_to_complex(x):
    return ComplexRI(x.numer / x.denom, 0)
coercions = {('rat', 'com') : rational_to_complex}

In [100]:
#这样我们便可以首先做类型转换然后仅仅调用运算符
coerce_apply_implementations = {('mul', 'com'): mul_complex,
                                ('mul', 'rat'): mul_rational,
                                ('add', 'com'): add_complex,
                                ('add', 'rat'): add_rational}
def coerce_apply(operator_name, x, y):
    tx, ty = type_tag(x), type_tag(y)
    if tx != ty:
        if (tx, ty) in coercions:
            tx, x = ty, coercions[(tx, ty)](x)
        elif (ty, tx) in coercions:
            ty, y = tx, coercions[(ty, tx)](y)
        else:
            raise KeyError('No coercion possible.')
    key = (operator_name, tx)
    return coerce_apply_implementations[key](x, y)

###### 总结
我们实现了 复数类 ComplexRI 和 ComplexMA, 以及有理数类Rational.而后实现了跨类型运算.
以加法运算为例, add_Complex 和 add_rational首先实现了同类之间的加法.然后我们实现了add_complex_and_rational方法实现两个不同类型的相加<br>
通过 type_tags和add_implementations分发字典, 我们实现了更为一般的add函数,他把不同的类型的组合映射到不同的函数中去.<br>
**我们可以看到add函数一般化了三个类的加法,我们有理由把他放到一个基类中去:**

In [3]:
def add_complex_and_rational(z, r):
    return ComplexRI(z.real + r.numer / r.denom, z.imag)
def add_rational_and_complex(r, z):
    return add_complex_and_rational(z, r)
class Number(object):
    adders = {
        ('com', 'rat') : add_complex_and_rational,
        ('rat', 'com') : add_rational_and_complex
    }
    def __add__(self, other):
        if self.type_tag == other.type_tag:
            return self.add(other)
        elif (self.type_tag, other.type_tag) in self.adders:
            return self.cross_apply(other, self.adders)
    def cross_apply(self, other, cross_func):
        cross_function = cross_func[(self.type_tag, other.type_tag)]
        return cross_function(self, other)

In [4]:
#我们的数类可以继承Number Mixin, 例如Rationsl
class Rational(Number):
    type_tag = 'rat'
    def __init__(self, numer, denom):
        g = gcd(numer, denom)
        self.numer = numer // g
        self.denom = denom // g
    def __repr__(self):
        pass
    #在子类中实现add方法, zhegeadd方法将在基类的__add__魔法函数中调用
    def add(self, other):
        nx, dx = self.numer, self.denom
        ny, dy = other.numer, other.denom
        return Rational(nx * dy + ny * dx, dx * dy)